In [2]:
from data_processing import *
import tensorflow as tf
import copy
import numpy as np

In [3]:
dialogues, ides = [], []
for i in range(0, 9):
    dialogues.append(preprocessed_dialogue(i))
    ides.append(preprocessed_ide(i))

In [4]:
dialogue_input, dialogue_output = sort(dialogues)
ide_input, ide_output = sort(ides)
maxLength = 0

# Max consecutive dialogue by one speaker
for i in dialogue_input:
    maxLength = max(maxLength, len(i))

In [6]:
one_hot_encoded_dialogue_input, one_hot_encoded_dialogue_output = np.array([]), np.array([])
sample = []
# padding dialogues up to max consecutive dialogue length
for i in dialogue_input:
    temp = i
    while (len(temp) < 21): temp.append(0)
    sample.append(temp)
    
from sklearn.preprocessing import MinMaxScaler
# scaler = MinMaxScaler()
# print(sample[3])
# sample = scaler.fit_transform(sample)
# print(sample[3])

one_hot_encoded_dialogue_input = tf.keras.utils.to_categorical(sample)


for i in dialogue_output:
    temp = tf.keras.utils.to_categorical(i).tolist()
    while (len(temp) < 47):
        temp.append(0)
    one_hot_encoded_dialogue_output = np.append(one_hot_encoded_dialogue_output, temp)
    
print(len(one_hot_encoded_dialogue_input), len(one_hot_encoded_dialogue_output))

2080 89440


In [9]:
from sklearn.metrics import confusion_matrix
def build_model():
    model = tf.keras.Sequential([
        tf.keras.layers.Dense(units=21, activation='relu'),
        tf.keras.layers.LSTM(units=128, input_shape=(21, 43)),
        tf.keras.layers.Dense(units=21, activation='relu'),
        tf.keras.layers.Dense(units=1, activation='relu')
    ])
    model.compile(optimizer='adam', loss='mean_squared_error', metrics=['accuracy'])
    return model

def train_and_evaluate_model(model, X, y, k=5):
    from sklearn.model_selection import KFold
    kfold = KFold(n_splits=k, shuffle=True, random_state=0)
    scores = []
    for train_index, test_index in kfold.split(X):
        X_train, X_test = X[train_index], X[test_index]
        y_train, y_test = y[train_index], y[test_index]
        model.fit(X_train, y_train, epochs=10, batch_size=32, verbose=0)
        score = model.evaluate(X_test, y_test, verbose=2)[1]
        print(score)
        scores.append(score)
    return np.mean(scores)

# print()
# Load the input data and one-hot encode it
X = one_hot_encoded_dialogue_input
y = np.array(dialogue_output)

model = build_model()
# model.summary()
score = train_and_evaluate_model(model, X, y)
print("Cross-validated accuracy:", score)


13/13 - 0s - loss: 45.5869 - accuracy: 0.2620 - 400ms/epoch - 31ms/step
0.26201921701431274
13/13 - 0s - loss: 47.3970 - accuracy: 0.2524 - 62ms/epoch - 5ms/step
0.2524038553237915
13/13 - 0s - loss: 52.8881 - accuracy: 0.3101 - 179ms/epoch - 14ms/step
0.3100961446762085
13/13 - 0s - loss: 69.8904 - accuracy: 0.2572 - 75ms/epoch - 6ms/step
0.2572115361690521
13/13 - 0s - loss: 60.0640 - accuracy: 0.2668 - 186ms/epoch - 14ms/step
0.26682692766189575
Cross-validated accuracy: 0.26971153616905214


In [13]:
y_prediction = model.predict(X)
for i in y_prediction: print(i)
result = confusion_matrix(y, y_prediction, normalize='pred')

65/65 [==============================] - 0s 5ms/step
[7.7874527]
[7.7874427]
[7.787612]
[7.7870574]
[7.7863407]
[7.7876167]
[7.7873616]
[7.787426]
[7.7874427]
[7.7870946]
[7.787415]
[7.7876167]
[7.7869897]
[7.7874427]
[7.7873254]
[7.7874427]
[7.785466]
[7.7874427]
[7.787426]
[7.787426]
[7.7874427]
[7.787426]
[7.7862434]
[7.7874427]
[7.787426]
[7.7873363]
[7.787358]
[7.7870946]
[7.78718]
[7.7874427]
[7.7870793]
[7.7874427]
[7.7858443]
[7.787544]
[7.7865496]
[7.7873416]
[7.7874427]
[7.787538]
[7.787252]
[7.787257]
[7.7876167]
[7.787426]
[7.7874427]
[7.787426]
[7.7868853]
[7.7869024]
[7.7876167]
[7.787426]
[7.787426]
[7.7869897]
[7.787426]
[7.785466]
[7.786366]
[7.7873964]
[7.787119]
[7.787426]
[7.787426]
[7.787426]
[7.787426]
[7.787426]
[7.787426]
[7.787426]
[7.787567]
[7.7876472]
[7.7876086]
[7.7874427]
[7.7872066]
[7.787567]
[7.786413]
[7.787426]
[7.7874427]
[7.787415]
[7.787426]
[7.7875314]
[7.786722]
[7.787447]
[7.7692447]
[7.7874427]
[7.787426]
[7.787378]
[7.7874427]
[7.7874527]
[7.

[7.787426]
[7.7874427]
[7.786366]
[7.7876086]
[7.7874427]
[7.7855906]
[7.7869897]
[7.7869897]
[7.787426]
[7.7864685]
[7.7874427]
[7.7868958]
[7.787426]
[7.786487]
[7.7874427]
[7.7874427]
[7.7870007]
[7.7870007]
[7.787426]
[7.7870283]
[7.7874427]
[7.7869897]
[7.78731]
[7.7874427]
[7.787426]
[7.787426]
[7.787415]
[7.787426]
[7.787426]
[7.7821126]
[7.787426]
[7.787415]
[7.787415]
[7.787426]
[7.7876086]
[7.779654]
[7.787426]
[7.786366]
[7.7874427]
[7.787426]
[7.7868824]
[7.7874427]
[7.786912]
[7.7863708]
[7.787426]
[7.787426]
[7.7873964]
[7.7874427]
[7.78731]
[7.785477]
[7.787426]
[7.785466]
[7.787426]
[7.786799]
[7.787189]
[7.7876167]
[7.7871594]
[7.786554]
[7.786514]
[7.78731]
[7.7876267]
[7.78731]
[7.7870574]
[7.787608]
[7.7876267]
[7.787426]
[7.787051]
[7.7874427]
[7.7870007]
[7.7874427]
[7.786366]
[7.7874427]
[7.7869897]
[7.785371]
[7.7873964]
[7.7863903]
[7.785466]
[7.787426]
[7.787426]
[7.787567]
[7.787426]
[7.7869897]
[7.787426]
[7.78731]
[7.787426]
[7.78731]
[7.7874427]
[7.787426]

ValueError: Classification metrics can't handle a mix of multiclass and continuous targets

In [ ]:
TypeError: only integer scalar arrays can be converted to a scalar index
    